In [10]:
import json
import numpy as np
from sklearn.model_selection import train_test_split
from collections import Counter
import pandas as pd
import shutil
import os
import random

In [11]:
annotation_ICDAR17 = '/opt/ml/input/data/ICDAR17_Korean/ufo/train.json'
annotation_Upstage = '/opt/ml/input/data/Upstage/ufo/train.json'

with open(annotation_ICDAR17) as f1: 
    data_ICDAR_17 = json.load(f1)
    
with open(annotation_Upstage) as f2:     
    data_Upstage = json.load(f2)

In [12]:
data = dict()
data['images'] = data_ICDAR_17['images']
data['images'].update(data_Upstage['images'])

with open(f'/opt/ml/input/data/total/ufo/all.json', 'w') as make_file:
    json.dump(data, make_file, indent='\t')

In [13]:
def split_dataset(input_json, output_dir, val_ratio, random_seed=1):
    random.seed(random_seed)

    with open(input_json) as json_reader:
        dataset = json.load(json_reader)

    images = dataset['images']

    image_ids = list(images.keys())
    image_ids.sort()
    random.shuffle(image_ids)

    num_val = int(len(image_ids) * val_ratio)
    num_train = len(image_ids) - num_val

    image_ids_val, image_ids_train = set(image_ids[:num_val]), set(image_ids[num_val:])

    train_images = {img_id: images[img_id] for img_id  in image_ids_train}
    val_images = {img_id: images[img_id] for img_id  in image_ids_val}

    train_data = {'images': train_images}
    val_data = {'images': val_images}

    output_train_json = os.path.join(output_dir, 'ufo/train.json')
    output_val_json = os.path.join(output_dir, 'ufo/val.json')

    print(f'write {output_train_json}')
    with open(output_train_json, 'w') as train_writer:
        json.dump(train_data, train_writer)

    print(f'write {output_val_json}')
    with open(output_val_json, 'w') as val_writer:
        json.dump(val_data, val_writer)

In [14]:
train_dir = '/opt/ml/input/data/total/ufo/'
val_dir = '/opt/ml/input/data/total/ufo/'

split_dataset(input_json='/opt/ml/input/data/total/ufo/all.json',
              output_dir='/opt/ml/input/data/total/',
              val_ratio=0.2,
              random_seed=1)

write /opt/ml/input/data/total/ufo/train.json
write /opt/ml/input/data/total/ufo/val.json
